# Dataset BioASQ

In [1]:
import json
import sys
from bs4 import BeautifulSoup
import requests
import sys,os,os.path
import re
import spacy
import numpy as np
import random
import seaborn as sns
import tqdm
import pprint

In [8]:
GOLD_PATH = 'gold_standard/'
files = [(f"{GOLD_PATH}BioASQ-trainingDataset{i}b.json", i) for i in [4, 5, 6, 7, 8]]
files

[('gold_standard/BioASQ-trainingDataset4b.json', 4),
 ('gold_standard/BioASQ-trainingDataset5b.json', 5),
 ('gold_standard/BioASQ-trainingDataset6b.json', 6),
 ('gold_standard/BioASQ-trainingDataset7b.json', 7),
 ('gold_standard/BioASQ-trainingDataset8b.json', 8)]

In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
pp = pprint.PrettyPrinter(indent=4)

### Estructura de `bioasq.question`
- `triplets`:
    - `o`: 'CRS',
    - `p`: 'http://www.w3.org/2008/05/skos-xl#literalForm',
    - `s`: 'http://linkedlifedata.com/resource/umls/label/A11993001'},

{
'pair_id': idq_docid_startchar-endchar,
'question_id',
'question',
'passage',
'trainset',
'doc_url',
'section': ['title','abstract']
'is_answer': 1,
'doc_related': 0,
'biosentvec_cossim': [0,1],
'offsetInBeginSection': 235,
'offsetInEndSection': 345,
}

In [21]:
def get_qa_true_pairs(question, trainset):
    '''Transform a BioASQ gold object into a QAPair object.'''
    def get_pair_id(question, answer):
        params = [
            question['id'], # question id
            answer['document'].split('/')[-1], # answer id
            answer['beginSection'],
            str(answer['offsetInBeginSection']),
            answer['endSection'],
            str(answer['offsetInEndSection'])
        ]
            
        return '-'.join(params)
    
#     pp.pprint(question)
    pairs = []
    for answer in question['snippets']:
        pair = {}
        pair['pairId'] = get_pair_id(question, answer)
        pair['questionId'] = question['id']
        pair['passage'] = answer['text']
        pair['trainset'] = f"{trainset}b"
        pair['doc_url'] = answer['document']
        pair['is_answer'] = 1
        pair['doc_related'] = 0
        pair['bioSentVecCosSim'] = 0
        pair['beginSection'] = answer['beginSection']
        pair['offsetInBeginSection'] = answer['offsetInBeginSection']
        pair['endSection'] = answer['endSection']
        pair['offsetInEndSection'] = answer['offsetInEndSection']
        
        pairs.append(pair)
    return pairs

In [22]:
true_pairs = []

for (file, trainset) in files[:1]:
    bioasq = json.load(open(file, "r"))
    for question in bioasq['questions'][:1]:
        true_pairs.extend(get_qa_true_pairs(question, trainset))
        
pp.pprint(true_pairs[0])

{   'beginSection': 'abstract',
    'bioSentVecCosSim': 0,
    'doc_related': 0,
    'doc_url': 'http://www.ncbi.nlm.nih.gov/pubmed/23378035',
    'endSection': 'abstract',
    'is_answer': 1,
    'offsetInBeginSection': 164,
    'offsetInEndSection': 481,
    'pairId': '52bf1d3c03868f1b0600000d-23378035-abstract-164-abstract-481',
    'passage': 'Muenke syndrome is characterized by coronal craniosynostosis '
               '(bilateral more often than unilateral), hearing loss, '
               'developmental delay, and carpal and/or tarsal bone coalition. '
               'Tarsal coalition is a distinct feature of Muenke syndrome and '
               'has been reported since the initial description of the '
               'disorder in the 1990s. ',
    'questionId': '52bf1d3c03868f1b0600000d',
    'trainset': '4b'}
